# Mapping the Wikidata relations to CSKG relations

## Map relations and Prepare for CSKG

In [77]:
new_columns=['node1', 'relation', 'node2', 'node1;label', 'node2;label','relation;label', 'relation;dimension', 'source', 'sentence']

In [78]:
tmp_dir='tmp/wikidata20200504'
#tmp_dir='tmp/wikidata20171227'#
tmp_dir='tmp/wikidata20181210'#
wd_file='%s/wikidata_cs.tsv' % tmp_dir
mapped='%s/kgtk_wikidata.tsv' % tmp_dir
stats='%s/stats_mapped.tsv' % tmp_dir

In [79]:
import pandas as pd
import mapping

In [80]:
df1=pd.read_csv(wd_file, sep='\t')

In [81]:
len(df1)

145956

In [82]:
mapped_fw=set()
mapped_bw=set()
rows=[]
for i, row in df1.iterrows():
    node1=row[0]
    rel=row[1]
    node2=row[2]
    node1_label=row[3].strip()[1:-4]
    node2_label=row[4].strip()[1:-4]
    rel_label=row[5].strip()[1:-4]
    if rel in mapping.fw_mapping.keys() :
        cn_rel=mapping.fw_mapping[rel]
        my_row=[node1,cn_rel,node2,node1_label,node2_label,rel_label, '', 'WD', '']
    elif rel in mapping.bw_mapping.keys():
        cn_rel=mapping.bw_mapping[rel]
        my_row=[node2,cn_rel,node1,node2_label,node1_label,rel_label, '', 'WD', '']
    rows.append(my_row)


In [83]:
df2=pd.DataFrame(rows, columns=new_columns)

In [84]:
df2.drop_duplicates(inplace=True)

In [85]:
len(df2)

145881

In [86]:
df2['relation'].value_counts()

/r/IsA                98760
/r/PartOf             13062
/r/HasContext          8029
/r/HasProperty         4505
/r/HasPrerequisite     4149
/r/UsedFor             3540
/r/RelatedTo           3479
/r/DistinctFrom        3214
/r/Antonym             2178
/r/Causes              1482
/r/MadeOf              1229
/r/Synonym             1070
/r/SimilarTo            437
/r/DerivedFrom          289
/r/CreatedBy            272
/r/AtLocation           133
/r/r/CreatedBy           53
Name: relation, dtype: int64

In [87]:
df2.to_csv(mapped, index=False, sep='\t', columns=new_columns)

In [88]:
%%bash -s "$mapped" "$stats"
kgtk graph_statistics --directed --degrees --pagerank --hits --log summary.txt -i $1 > $2

In [89]:
%%bash
cat summary.txt

loading the TSV graph now ...
graph loaded! It has 76066 nodes and 145881 edges

###Top relations:
/r/IsA	98760
/r/PartOf	13062
/r/HasContext	8029
/r/HasProperty	4505
/r/HasPrerequisite	4149
/r/UsedFor	3540
/r/RelatedTo	3479
/r/DistinctFrom	3214
/r/Antonym	2178
/r/Causes	1482

###Degrees:
in degree stats: mean=1.917821, std=0.102828, max=1
out degree stats: mean=1.917821, std=0.006155, max=1
total degree stats: mean=3.835643, std=0.103883, max=1

###PageRank
Max pageranks
2502	Q3249551	0.006873
386	Q11862829	0.009361
288	Q151885	0.010839
310	Q2996394	0.008839
1906	Q7187	0.010410

###HITS
HITS hubs
36441	Q62536	0.000000
310	Q2996394	0.000000
2683	Q8054	0.000281
14572	Q20747295	0.000770
1906	Q7187	1.000000
HITS auth
14572	Q20747295	0.013873
70327	Q20969150	0.013880
37064	Q18256813	0.013880
49434	Q18969817	0.013880
14571	Q26738515	0.013880
